In [1]:


tf.__version__

'2.0.0'

### Based on the implementation of [Spinning Up](https://spinningup.openai.com/en/latest/spinningup/extra_tf_pg_implementation.html)

In [73]:
import numpy as np

import gym
from gym.spaces import Discrete, Box

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
import tensorflow.keras.optimizers as optimizers


class mlp(keras.Model):
    def __init__(self, hidden_size, obs_input, num_actions):
        super().__init__()
        self.hidden1 = layers.Dense(hidden_size, activation=tf.tanh)
        self.hidden2 = layers.Dense(hidden_size, activation=tf.tanh)
        self.logits = layers.Dense(num_actions, name='policy_logits')
        
    def call(self, x, **kwargs):
        x = tf.convert_to_tensor(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        return self.logits(x)

    def actions(self, obs):
        logits = self.predict(obs)
        actions = tf.squeeze(tf.multinomial(logits=logits, num_samples=1), axis=1)
        return actions


class PolicyGradient:
    # TODO: Define more hyperparameters
    def __init__(self, env_name='CartPole-v0', 
                 hidden_size=32, lr=1e-2):
        
        # make environment, check spaces, get obs / act dims
        self.env = gym.make(env_name)
        assert isinstance(env.observation_space, Box), \
            "This example only works for envs with continuous state spaces."
        assert isinstance(env.action_space, Discrete), \
            "This example only works for envs with discrete action spaces."

        self.obs_dim = env.observation_space.shape[0]
        self.n_acts = env.action_space.n
        
        # TODO: Possibly pull MLP definition from init
        self.model = mlp(hidden_size=hidden_size, obs_input=obs_dim, num_actions=n_acts)
        self.model.compile(
          optimizer=optimizers.Adam(lr=lr),
          loss=[self._loss])

    def _loss(self, obs, act, weights):
        logits = self.model.predict(obs)
        action_masks = tf.one_hot(act.shape[0], logits.shape[1])
        log_probs = tf.reduce_sum(action_masks * tf.nn.log_softmax(logits), axis=1)
        loss = -tf.reduce_mean(weights * log_probs)
        return loss
    
    def train(self, epochs=50, batch_size=64, render=False):
        # training loop
        for i in range(epochs):
            batch_loss, batch_rets, batch_lens = self.train_one_epoch()
            print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
                    (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))
        
    # for training policy
    def train_one_epoch(self, render=False):
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for R(tau) weighting in policy gradient
        batch_rets = []         # for measuring episode returns
        batch_lens = []         # for measuring episode lengths

        # reset episode-specific variables
        obs = env.reset()       # first obs comes from starting distribution
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep

        # render first episode of each epoch
        finished_rendering_this_epoch = False

        # collect experience by acting in the environment with current policy
        while True:

            # rendering
            if (not finished_rendering_this_epoch) and render:
                env.render()

            # save obs
            batch_obs.append(obs.copy())

            # act in the environment
            # obs.reshape(1, -1)?
            act = self.model.actions(obs.reshape(1, -1))[0]
            obs, rew, done, _ = env.step(act)

            # save action, reward
            batch_acts.append(act)
            ep_rews.append(rew)

            if done:
                # if episode is over, record info about episode
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)
                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)

                # the weight for each logprob(a|s) is R(tau)
                batch_weights += [ep_ret] * ep_len

                # reset episode-specific variables
                obs, done, ep_rews = env.reset(), False, []

                # won't render again this epoch
                finished_rendering_this_epoch = True

                # end experience loop if we have enough of it
                if len(batch_obs) > batch_size:
                    break

        
        losses = self.model.train_on_batch(obs)
        return losses
#         # take a single policy gradient update step
#         batch_loss, _ = sess.run([loss, train_op],
#                                  feed_dict={
#                                     obs_ph: np.array(batch_obs),
#                                     act_ph: np.array(batch_acts),
#                                     weights_ph: np.array(batch_weights)
#                                  })
#         return batch_loss, batch_rets, batch_lens

In [74]:
pg = PolicyGradient()

pg.train()

TypeError: _loss() missing 1 required positional argument: 'weights'

In [60]:
import tensorflow as tf
import numpy as np
import gym
from gym.spaces import Discrete, Box



def train(env_name='CartPole-v0', hidden_sizes=[32], lr=1e-2, 
          epochs=50, batch_size=5000, render=False):

    # make environment, check spaces, get obs / act dims
    env = gym.make(env_name)
    assert isinstance(env.observation_space, Box), \
        "This example only works for envs with continuous state spaces."
    assert isinstance(env.action_space, Discrete), \
        "This example only works for envs with discrete action spaces."

    obs_dim = env.observation_space.shape[0]
    n_acts = env.action_space.n

    # make core of policy network
    obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
    logits = mlp(obs_ph, sizes=hidden_sizes+[n_acts])

    # make action selection op (outputs int actions, sampled from policy)
    actions = tf.squeeze(tf.multinomial(logits=logits, num_samples=1), axis=1)

    # make loss function whose gradient, for the right data, is policy gradient
    weights_ph = tf.placeholder(shape=(None,), dtype=tf.float32)
    act_ph = tf.placeholder(shape=(None,), dtype=tf.int32)
    action_masks = tf.one_hot(act_ph, n_acts)
    log_probs = tf.reduce_sum(action_masks * tf.nn.log_softmax(logits), axis=1)
    loss = -tf.reduce_mean(weights_ph * log_probs)

    # make train op
    train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    # for training policy
    def train_one_epoch():
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for R(tau) weighting in policy gradient
        batch_rets = []         # for measuring episode returns
        batch_lens = []         # for measuring episode lengths

        # reset episode-specific variables
        obs = env.reset()       # first obs comes from starting distribution
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep

        # render first episode of each epoch
        finished_rendering_this_epoch = False

        # collect experience by acting in the environment with current policy
        while True:

            # rendering
            if (not finished_rendering_this_epoch) and render:
                env.render()

            # save obs
            batch_obs.append(obs.copy())

            # act in the environment
            act = sess.run(actions, {obs_ph: obs.reshape(1,-1)})[0]
            obs, rew, done, _ = env.step(act)

            # save action, reward
            batch_acts.append(act)
            ep_rews.append(rew)

            if done:
                # if episode is over, record info about episode
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)
                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)

                # the weight for each logprob(a|s) is R(tau)
                batch_weights += [ep_ret] * ep_len

                # reset episode-specific variables
                obs, done, ep_rews = env.reset(), False, []

                # won't render again this epoch
                finished_rendering_this_epoch = True

                # end experience loop if we have enough of it
                if len(batch_obs) > batch_size:
                    break

        # take a single policy gradient update step
        batch_loss, _ = sess.run([loss, train_op],
                                 feed_dict={
                                    obs_ph: np.array(batch_obs),
                                    act_ph: np.array(batch_acts),
                                    weights_ph: np.array(batch_weights)
                                 })
        return batch_loss, batch_rets, batch_lens

    # training loop
    for i in range(epochs):
        batch_loss, batch_rets, batch_lens = train_one_epoch()
        print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
                (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))

train()